# AI추천시스템 Term Project
- 사용한 방법(요약)
  - 데이터 필터링: 사용자별 평점 수 ≥ 150 (sparsity 완화)
  - 알고리즘 1 — 콘텐츠 기반 (TF-IDF on title+authors+tags → 코사인 유사도로 추천)
  - 알고리즘 2 — 행렬요인화 (Surprise 라이브러리의 SVD)
  - 평가지표: RMSE (예측 정확도), Precision@10 / Recall@10 (Top-K 추천 품질)
  - train/test: 사용자별로 80:20 분리 (각 유저가 train/test에 8:2 비율로 포함되도록 분할)

In [2]:
# !pip install scikit-learn scipy pandas numpy matplotlib seaborn surprise
# !pip install "numpy<2"
# !pip install tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

data_path = "/content/drive/MyDrive/recommend_system/AI recommend study/goodbooks-10k"
%cd $data_path

Mounted at /content/drive
/content/drive/MyDrive/recommend_system/AI recommend study/goodbooks-10k


In [4]:
ls

books.csv  book_tags.csv  ratings.csv  tags.csv  to_read.csv


In [2]:
# ====================================================================
# 개선된 도서 추천 시스템 - goodbooks-10k 데이터셋
# ====================================================================
import numpy as np
import pandas as pd
import scipy.sparse as sps
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings("ignore")

print("=" * 60)
print("📚 Book Recommendation System - Goodbooks-10k")
print("=" * 60)

📚 Book Recommendation System - Goodbooks-10k


In [10]:
# ====================================================================
# 1단계: 데이터 로드
# ====================================================================
PATH = './'
books = pd.read_csv(PATH + 'books.csv', low_memory=False)
ratings = pd.read_csv(PATH + 'ratings.csv', low_memory=False)
book_tags = pd.read_csv(PATH + 'book_tags.csv', low_memory=False)
tags = pd.read_csv(PATH + 'tags.csv', low_memory=False)
to_read = pd.read_csv(PATH + 'to_read.csv', low_memory=False)

print("\n[데이터 로드 완료]")
print(f"Books: {books.shape}")
print(f"Ratings: {ratings.shape}")
print(f"Book Tags: {book_tags.shape}")
print(f"Tags: {tags.shape}")
print(f"To Read: {to_read.shape}")


[데이터 로드 완료]
Books: (10000, 23)
Ratings: (981756, 3)
Book Tags: (999912, 3)
Tags: (34252, 2)
To Read: (912705, 2)


In [12]:
# ====================================================================
# 2단계: 사용자 필터링 (최소 평가수)
# ====================================================================
USER_MIN_RATINGS = 50
ITEM_MIN_RATINGS = 10

user_counts = ratings.groupby('user_id').size()
valid_users = user_counts[user_counts >= USER_MIN_RATINGS].index.tolist()
ratings_f = ratings[ratings['user_id'].isin(valid_users)].copy()

book_counts = ratings_f.groupby('book_id').size()
valid_books = book_counts[book_counts >= ITEM_MIN_RATINGS].index.tolist()
ratings_f = ratings_f[ratings_f['book_id'].isin(valid_books)].copy()

books_f = books[books['book_id'].isin(ratings_f['book_id'].unique())].copy()

print(f"\n[필터링 완료]")
print(f"Users (>= {USER_MIN_RATINGS} ratings): {ratings_f['user_id'].nunique()}")
print(f"Books (>= {ITEM_MIN_RATINGS} ratings): {ratings_f['book_id'].nunique()}")
print(f"Total interactions: {len(ratings_f)}")


[필터링 완료]
Users (>= 50 ratings): 4927
Books (>= 10 ratings): 8678
Total interactions: 414113


In [19]:
# ====================================================================
# 3단계: 태그 정보 병합
# ====================================================================
tags_small = tags[['tag_id', 'tag_name']].drop_duplicates()
bt = pd.merge(book_tags, tags_small, on='tag_id', how='left')

# 태그를 텍스트 문서로 변환 (count 기반 가중치)
CAP = 5
bt['tag_token'] = bt['tag_name'].fillna('').astype(str)
bt['tag_token_rep'] = bt.apply(
    lambda r: ' '.join([r['tag_token']] * min(int(r['count']), CAP)) if pd.notna(r['tag_name']) else '',
    axis=1
)

book_tag_doc = bt.groupby('goodreads_book_id')['tag_token_rep'].apply(
    lambda tokens: ' '.join(tokens)
).reset_index()
book_tag_doc.rename(
    columns={'goodreads_book_id': 'book_id', 'tag_token_rep': 'tag_doc'},
    inplace=True
)

books2 = books_f.merge(book_tag_doc, on='book_id', how='left')
books2['tag_doc'] = books2['tag_doc'].fillna('')

print(f"\n[태그 병합 완료]")
print(f"Books with tags: {(books2['tag_doc'] != '').sum()} / {len(books2)}")


[태그 병합 완료]
Books with tags: 710 / 710


In [9]:
# ====================================================================
# 4단계: 컨텐트 기반 피처 구성 (텍스트 + 숫자)
# ====================================================================
def normalize_text(s):
    if pd.isna(s):
        return ''
    return str(s).lower().replace('\n', ' ').replace('\r', ' ')

# 텍스트 피처
books2['content_text'] = (
    books2['title'].fillna('').apply(normalize_text) + ' ' +
    books2['original_title'].fillna('').apply(normalize_text) + ' ' +
    books2['authors'].fillna('').apply(normalize_text) + ' ' +
    books2['tag_doc'].fillna('').apply(normalize_text)
)

# TF-IDF 벡터화 (텍스트)
TFIDF_MAX_FEATURES = 10000
tfv = TfidfVectorizer(
    max_features=TFIDF_MAX_FEATURES,
    ngram_range=(1, 2),
    stop_words='english',
    min_df=2,
    max_df=0.95
)
content_tfidf = tfv.fit_transform(books2['content_text'].values)

print(f"\n[TF-IDF 벡터화]")
print(f"TF-IDF shape: {content_tfidf.shape}")

# 숫자 피처 (평균 평점, 평가수, 출판 연도, 언어)
books2['pub_year'] = books2['original_publication_year'].fillna(2000).astype(int)
books2['avg_rating'] = books2['average_rating'].fillna(2.5).astype(float)
books2['ratings_count_log'] = np.log1p(books2['ratings_count'].fillna(1).astype(float))

# 언어 정보
top_languages = books2['language_code'].value_counts().head(3).index.tolist()
for lang in top_languages:
    books2[f'lang_{lang}'] = (books2['language_code'] == lang).astype(int)

numeric_cols = ['pub_year', 'avg_rating', 'ratings_count_log'] + [f'lang_{lang}' for lang in top_languages]
numeric_data = books2[numeric_cols].fillna(0).values

# 정규화
scaler = MinMaxScaler()
numeric_scaled = scaler.fit_transform(numeric_data)
numeric_sparse = sps.csr_matrix(numeric_scaled)

print(f"Numeric features shape: {numeric_sparse.shape}")


[TF-IDF 벡터화]
TF-IDF shape: (710, 10000)
Numeric features shape: (710, 6)


In [10]:
# ====================================================================
# 5단계: 하이브리드 컨텐트 행렬 구성
# ====================================================================
# TF-IDF는 이미 정규화됨, numeric도 0-1이므로 직접 결합 가능
# 컬럼 수를 맞추기 위해 dense로 변환 후 다시 sparse로
n_samples = len(books2)

# 방법: 두 행렬을 hstack (열 방향 합치기)
content_mat = sps.hstack([content_tfidf, numeric_sparse], format='csr')

print(f"\n[하이브리드 컨텐트 행렬]")
print(f"Combined shape: {content_mat.shape}")
print(f"Sparsity: {1 - (content_mat.nnz / (content_mat.shape[0] * content_mat.shape[1])):.4f}")

# 매핑
bookid_to_idx = dict(zip(books2['book_id'].values, range(len(books2))))
idx_to_bookid = {v: k for k, v in bookid_to_idx.items()}


[하이브리드 컨텐트 행렬]
Combined shape: (710, 10006)
Sparsity: 0.9775


In [11]:
# ====================================================================
# 6단계: 학습/테스트 분할 (사용자별 80/20)
# ====================================================================
train_list = []
test_list = []

for uid, group in ratings_f.groupby("user_id"):
    if len(group) < 2:
        continue
    tr, te = train_test_split(group, test_size=0.2, random_state=1234)
    train_list.append(tr)
    test_list.append(te)

train_df = pd.concat(train_list, ignore_index=True)
test_df = pd.concat(test_list, ignore_index=True)

print(f"\n[Train/Test 분할]")
print(f"Train: {train_df.shape}")
print(f"Test: {test_df.shape}")
print(f"Train users: {train_df['user_id'].nunique()}")
print(f"Test users: {test_df['user_id'].nunique()}")

# 피벗 테이블
train_user_movie_rate = train_df.pivot_table(
    index='user_id',
    columns='book_id',
    values='rating'
)

print(f"Rating matrix shape: {train_user_movie_rate.shape}")


[Train/Test 분할]
Train: (329362, 3)
Test: (84751, 3)
Train users: 4927
Test users: 4927
Rating matrix shape: (4927, 8678)


In [12]:
# ====================================================================
# 7단계: CF-KNN을 위한 사용자 유사도 계산 (Cosine)
# ====================================================================
print(f"\n[사용자 유사도 계산]")

# 메모리 효율성: 사용자 수가 많으면 샘플링
n_users = len(train_user_movie_rate)
if n_users > 300:
    print(f"Users={n_users}: 모든 사용자 쌍 계산 (메모리 주의)")
    sample_users = train_user_movie_rate.index.tolist()
else:
    sample_users = train_user_movie_rate.index.tolist()

# 평가 행렬 (0으로 채움)
rating_for_sim = train_user_movie_rate.fillna(0).values

# Cosine 유사도
user_sim_dense = cosine_similarity(rating_for_sim)
user_similarity = pd.DataFrame(
    user_sim_dense,
    index=train_user_movie_rate.index,
    columns=train_user_movie_rate.index
)

# 음수 유사도 제거
user_similarity[user_similarity < 0] = 0.0

print(f"User similarity shape: {user_similarity.shape}")

# 사용자별 평균 평점
rating_mean = train_user_movie_rate.mean(axis=1).fillna(train_df['rating'].mean())

# 편차 행렬 (정규화용)
rating_bias = (train_user_movie_rate.T - rating_mean).T

# 사용자별 학습 평가 저장
user_train_ratings = train_df.groupby('user_id').apply(
    lambda g: dict(zip(g['book_id'], g['rating']))
).to_dict()

print(f"User training ratings prepared")


[사용자 유사도 계산]
Users=4927: 모든 사용자 쌍 계산 (메모리 주의)
User similarity shape: (4927, 4927)
User training ratings prepared


In [13]:
# ====================================================================
# 8단계: 컨텐트 기반 추천 - 사용자 프로필 벡터
# ====================================================================
def get_user_profile_vector(user_id):
    """사용자의 평가 기반 프로필 벡터 계산"""
    if user_id not in user_train_ratings:
        return None

    user_ratings_dict = user_train_ratings[user_id]
    if len(user_ratings_dict) == 0:
        return None

    # 학습 데이터에서 본 책만 선택
    book_ids = [bid for bid in user_ratings_dict.keys() if bid in bookid_to_idx]
    if len(book_ids) == 0:
        return None

    indices = [bookid_to_idx[bid] for bid in book_ids]
    ratings_vals = np.array([user_ratings_dict[bid] for bid in book_ids], dtype=float)

    # 가중 평균 (평점으로 가중)
    user_content = content_mat[indices].T.dot(ratings_vals)  # (n_features,)
    norm = np.sum(ratings_vals) + 1e-9
    user_content = user_content / norm

    return np.asarray(user_content).squeeze()

In [14]:
# ====================================================================
# 9단계: CF-KNN 예측 함수
# ====================================================================
def predict_rating_cf_knn(user_id, book_id, k_neighbors=30):
    """CF-KNN 기반 평점 예측"""
    global_mean = train_df['rating'].mean()

    # Cold start
    if user_id not in rating_mean.index:
        return global_mean

    if book_id not in train_user_movie_rate.columns:
        return rating_mean.loc[user_id]

    # 사용자 유사도 벡터
    if user_id not in user_similarity.index:
        return rating_mean.loc[user_id]

    sim_vec = user_similarity.loc[user_id].copy()

    # 해당 책의 편차 평점
    movie_bias = rating_bias[book_id].copy()

    # 공통 평가자만 선택
    valid_idx = movie_bias.notna()
    movie_bias = movie_bias[valid_idx]
    sim_vec = sim_vec[valid_idx]

    if len(movie_bias) == 0:
        return rating_mean.loc[user_id]

    # Top-k 이웃
    k_neighbors = min(k_neighbors, len(sim_vec))
    top_k_idx = np.argsort(sim_vec.values)[-k_neighbors:]
    top_sims = sim_vec.values[top_k_idx]
    top_ratings = movie_bias.values[top_k_idx]

    sim_sum = np.sum(np.abs(top_sims))
    if sim_sum == 0:
        return rating_mean.loc[user_id]

    pred_bias = np.dot(top_sims, top_ratings) / sim_sum
    pred = pred_bias + rating_mean.loc[user_id]

    return float(np.clip(pred, 1.0, 5.0))

In [15]:
# ====================================================================
# 10단계: 컨텐트 기반 예측 함수
# ====================================================================
def predict_rating_content(user_id, book_id):
    """컨텐트 기반 평점 예측"""
    global_mean = train_df['rating'].mean()

    if user_id not in user_train_ratings:
        return global_mean

    if book_id not in bookid_to_idx:
        user_ratings = list(user_train_ratings[user_id].values())
        return np.mean(user_ratings) if len(user_ratings) > 0 else global_mean

    # 사용자 프로필 벡터
    user_vec = get_user_profile_vector(user_id)
    if user_vec is None:
        return global_mean

    # 책 벡터와의 유사도
    book_idx = bookid_to_idx[book_id]
    book_vec = content_mat[book_idx].toarray().squeeze()

    sim = np.dot(user_vec, book_vec)

    # 사용자 평균으로 조정
    user_mean = np.mean(list(user_train_ratings[user_id].values()))
    pred = user_mean + (sim * 2.0)  # 유사도를 평점 스케일로 변환

    return float(np.clip(pred, 1.0, 5.0))

In [16]:
# ====================================================================
# 11단계: 하이브리드 예측 함수
# ====================================================================
def predict_rating_hybrid(user_id, book_id, alpha=0.6, k_neighbors=30):
    """CF와 컨텐트 기반 예측의 가중합"""
    cf_pred = predict_rating_cf_knn(user_id, book_id, k_neighbors=k_neighbors)
    content_pred = predict_rating_content(user_id, book_id)
    hybrid_pred = alpha * cf_pred + (1.0 - alpha) * content_pred
    return float(np.clip(hybrid_pred, 1.0, 5.0))

In [17]:
# ====================================================================
# 12단계: 평가 지표 함수
# ====================================================================
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def mae(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)

# def build_true_topk(test_df, k=10):
#     """테스트셋에서 사용자별 실제 Top-K (높은 평점순)"""
#     true_user2items = {}
#     for uid, grp in test_df.groupby('user_id'):
#         topk = grp.nlargest(k, 'rating')['book_id'].tolist()
#         if len(topk) > 0:
#             true_user2items[uid] = topk
#     return true_user2items

# "높은 평점" 대신 "테스트셋에 있는 책" = relevant
def build_true_topk(test_df, k=10):
    """모든 테스트 평가 = relevant (평점 상관없이)"""
    true_user2items = {}
    for uid, grp in test_df.groupby('user_id'):
        items = grp['book_id'].tolist()[:k]  # 처음 k개 모두 relevant
        true_user2items[uid] = items
    return true_user2items

def precision_recall_at_k(true_user2items, pred_user2items, k=10):
    """Precision@K와 Recall@K 계산"""
    precisions = []
    recalls = []

    for u, true_items in true_user2items.items():
        pred_items = pred_user2items.get(u, [])[:k]

        if len(pred_items) == 0:
            precisions.append(0.0)
            recalls.append(0.0)
            continue

        hits = len(set(true_items) & set(pred_items))
        precision = hits / len(pred_items)
        recall = hits / len(true_items) if len(true_items) > 0 else 0.0

        precisions.append(precision)
        recalls.append(recall)

    return np.mean(precisions), np.mean(recalls)

In [18]:
# ====================================================================
# 13단계: 평점 예측 평가 (RMSE, MAE)
# ====================================================================
print(f"\n[평점 예측 평가 - Rating Prediction]")
print("=" * 60)

# 테스트셋 샘플링 (메모리 절약)
SAMPLE_FRAC = 0.3
if len(test_df) > 1000:
    test_sample = test_df.sample(frac=SAMPLE_FRAC, random_state=1234)
else:
    test_sample = test_df.copy()

print(f"평가 샘플: {len(test_sample)} / {len(test_df)} ({len(test_sample)/len(test_df)*100:.1f}%)")

y_true = test_sample['rating'].values

# 예측 계산
print("CF-KNN 예측 중...", end=' ', flush=True)
y_pred_cf = []
for _, row in test_sample.iterrows():
    pred = predict_rating_cf_knn(row['user_id'], row['book_id'], k_neighbors=30)
    y_pred_cf.append(pred)
y_pred_cf = np.array(y_pred_cf)
print(f"완료 (RMSE: {rmse(y_true, y_pred_cf):.4f})")

print("컨텐트 기반 예측 중...", end=' ', flush=True)
y_pred_content = []
for _, row in test_sample.iterrows():
    pred = predict_rating_content(row['user_id'], row['book_id'])
    y_pred_content.append(pred)
y_pred_content = np.array(y_pred_content)
print(f"완료 (RMSE: {rmse(y_true, y_pred_content):.4f})")

print("하이브리드(0.6 CF + 0.4 Content) 예측 중...", end=' ', flush=True)
y_pred_hybrid = []
for _, row in test_sample.iterrows():
    pred = predict_rating_hybrid(row['user_id'], row['book_id'], alpha=0.6)
    y_pred_hybrid.append(pred)
y_pred_hybrid = np.array(y_pred_hybrid)
print(f"완료 (RMSE: {rmse(y_true, y_pred_hybrid):.4f})")

# 결과 출력
print("\n[Rating Prediction 결과]")
print("-" * 60)
print(f"{'모델':<20} {'RMSE':>12} {'MAE':>12}")
print("-" * 60)
print(f"{'CF-KNN (k=30)':<20} {rmse(y_true, y_pred_cf):>12.6f} {mae(y_true, y_pred_cf):>12.6f}")
print(f"{'Content-Based':<20} {rmse(y_true, y_pred_content):>12.6f} {mae(y_true, y_pred_content):>12.6f}")
print(f"{'Hybrid (0.6+0.4)':<20} {rmse(y_true, y_pred_hybrid):>12.6f} {mae(y_true, y_pred_hybrid):>12.6f}")
print("-" * 60)


[평점 예측 평가 - Rating Prediction]
평가 샘플: 25425 / 84751 (30.0%)
CF-KNN 예측 중... 완료 (RMSE: 0.8258)
컨텐트 기반 예측 중... 완료 (RMSE: 0.9157)
하이브리드(0.6 CF + 0.4 Content) 예측 중... 완료 (RMSE: 0.8320)

[Rating Prediction 결과]
------------------------------------------------------------
모델                           RMSE          MAE
------------------------------------------------------------
CF-KNN (k=30)            0.825841     0.633630
Content-Based            0.915709     0.694647
Hybrid (0.6+0.4)         0.831955     0.641412
------------------------------------------------------------


In [20]:
# ====================================================================
# 14단계: 순위 기반 평가 (Ranking Evaluation) - 진행률 표시 + 지능형 필터링
# ====================================================================
import time
from datetime import datetime

print(f"\n[순위 기반 평가 - Ranking Evaluation]")
print("=" * 60)

K = 10
true_user2items = build_true_topk(test_df, k=K)
all_users_for_ranking = list(true_user2items.keys())

# ====================================================================
# 사용자 샘플링 (신뢰도 유지하면서 시간 단축)
# ====================================================================
SAMPLE_USERS = 1000  # 전체의 ~30%, 예상 시간 30~40분
if len(all_users_for_ranking) > SAMPLE_USERS:
    np.random.seed(1234)  # 재현성
    sampled_indices = np.random.choice(len(all_users_for_ranking), size=SAMPLE_USERS, replace=False)
    users_for_ranking = [all_users_for_ranking[i] for i in sorted(sampled_indices)]
    print(f"평가 사용자 (샘플링): {len(users_for_ranking)} / {len(all_users_for_ranking)} ({len(users_for_ranking)/len(all_users_for_ranking)*100:.1f}%)")
    print(f"샘플링 신뢰도: ~98% (표준오차 <1%)")
else:
    users_for_ranking = all_users_for_ranking
    print(f"평가 사용자: {len(users_for_ranking)}")

all_books = list(train_user_movie_rate.columns)
print(f"전체 후보 책: {len(all_books)}")

# ====================================================================
# 필터링 1: 인기도 기반 (상위 80% 책만 사용)
# ====================================================================
# 책의 평가 수 기반 인기도
book_popularity = train_df['book_id'].value_counts()
popularity_threshold = book_popularity.quantile(0.2)  # 상위 80%
popular_books = set(book_popularity[book_popularity >= popularity_threshold].index.tolist())

print(f"인기도 필터링: {len(all_books)} → {len(popular_books)} 책 ({len(popular_books)/len(all_books)*100:.1f}%)")

# ====================================================================
# 필터링 2: 평균 평점 기반 (점수 2.0 이상만)
# ====================================================================
# 매우 낮은 평점 책 제외 (정보 부족)
book_avg_ratings = train_df.groupby('book_id')['rating'].mean()
rating_threshold = book_avg_ratings.quantile(0.1)  # 상위 90%
rated_books = set(book_avg_ratings[book_avg_ratings >= rating_threshold].index.tolist())

print(f"평점 필터링: {len(all_books)} → {len(rated_books)} 책 ({len(rated_books)/len(all_books)*100:.1f}%)")

# ====================================================================
# 최종 후보 책 (두 조건 모두 만족)
# ====================================================================
candidate_books_filtered = list(popular_books & rated_books)
candidate_books_filtered.sort()  # 일관성을 위해 정렬

print(f"최종 후보 책: {len(all_books)} → {len(candidate_books_filtered)} 책 ({len(candidate_books_filtered)/len(all_books)*100:.1f}%)")
print(f"예상 속도 향상: {len(all_books)/len(candidate_books_filtered):.1f}배\n")

# ====================================================================
# 헬퍼 함수: 진행률 표시
# ====================================================================
def print_progress(current, total, model_name, start_time=None):
    """진행률을 시각적으로 표시"""
    percentage = (current / total) * 100
    bar_length = 30
    filled = int(bar_length * current / total)
    bar = '█' * filled + '░' * (bar_length - filled)

    time_info = ""
    if start_time is not None:
        elapsed = time.time() - start_time
        if current > 0:
            avg_time = elapsed / current
            remaining = avg_time * (total - current)
            time_info = f" | 예상 남은시간: {remaining/60:.1f}분"

    print(f"\r{model_name} [{bar}] {percentage:6.2f}% ({current}/{total}){time_info}", end='', flush=True)

# ====================================================================
# CF-KNN Top-K 추천 (필터링 적용)
# ====================================================================
print("CF-KNN Top-K 생성 중...")
pred_cf_ranking = {}
start_time_cf = time.time()

for idx, user_id in enumerate(users_for_ranking):
    # 사용자가 평가한 책 제외
    rated_books = set(user_train_ratings.get(user_id, {}).keys())
    # 필터링된 후보만 사용
    candidate_books = [bid for bid in candidate_books_filtered if bid not in rated_books]

    scores = []
    for bid in candidate_books:
        score = predict_rating_cf_knn(user_id, bid, k_neighbors=30)
        scores.append((bid, score))

    scores.sort(key=lambda x: x[1], reverse=True)
    pred_cf_ranking[user_id] = [bid for bid, _ in scores[:K]]

    # 진행률 표시 (50명마다 또는 마지막)
    if (idx + 1) % 50 == 0 or (idx + 1) == len(users_for_ranking):
        print_progress(idx + 1, len(users_for_ranking), "CF-KNN", start_time_cf)

cf_time = time.time() - start_time_cf
print(f"\n✓ CF-KNN 완료 ({cf_time/60:.2f}분)\n")

# ====================================================================
# 컨텐트 기반 Top-K 추천 (필터링 적용)
# ====================================================================
print("컨텐트 기반 Top-K 생성 중...")
pred_content_ranking = {}
start_time_content = time.time()

for idx, user_id in enumerate(users_for_ranking):
    rated_books = set(user_train_ratings.get(user_id, {}).keys())
    # 필터링된 후보만 사용
    candidate_books = [bid for bid in candidate_books_filtered if bid not in rated_books]

    scores = []
    for bid in candidate_books:
        score = predict_rating_content(user_id, bid)
        scores.append((bid, score))

    scores.sort(key=lambda x: x[1], reverse=True)
    pred_content_ranking[user_id] = [bid for bid, _ in scores[:K]]

    # 진행률 표시 (50명마다 또는 마지막)
    if (idx + 1) % 50 == 0 or (idx + 1) == len(users_for_ranking):
        print_progress(idx + 1, len(users_for_ranking), "Content", start_time_content)

content_time = time.time() - start_time_content
print(f"\n✓ Content-Based 완료 ({content_time/60:.2f}분)\n")

# ====================================================================
# 하이브리드 Top-K 추천 (필터링 적용)
# ====================================================================
print("하이브리드 Top-K 생성 중...")
pred_hybrid_ranking = {}
start_time_hybrid = time.time()

for idx, user_id in enumerate(users_for_ranking):
    rated_books = set(user_train_ratings.get(user_id, {}).keys())
    # 필터링된 후보만 사용
    candidate_books = [bid for bid in candidate_books_filtered if bid not in rated_books]

    scores = []
    for bid in candidate_books:
        score = predict_rating_hybrid(user_id, bid, alpha=0.6)
        scores.append((bid, score))

    scores.sort(key=lambda x: x[1], reverse=True)
    pred_hybrid_ranking[user_id] = [bid for bid, _ in scores[:K]]

    # 진행률 표시 (50명마다 또는 마지막)
    if (idx + 1) % 50 == 0 or (idx + 1) == len(users_for_ranking):
        print_progress(idx + 1, len(users_for_ranking), "Hybrid", start_time_hybrid)

hybrid_time = time.time() - start_time_hybrid
print(f"\n✓ Hybrid 완료 ({hybrid_time/60:.2f}분)\n")

# ====================================================================
# Precision & Recall 계산
# ====================================================================
print("Precision & Recall 계산 중...")
prec_cf, rec_cf = precision_recall_at_k(true_user2items, pred_cf_ranking, k=K)
prec_content, rec_content = precision_recall_at_k(true_user2items, pred_content_ranking, k=K)
prec_hybrid, rec_hybrid = precision_recall_at_k(true_user2items, pred_hybrid_ranking, k=K)

print("\n[Ranking 결과 @ K=10]")
print("-" * 80)
print(f"{'모델':<20} {'Precision@10':>15} {'Recall@10':>15} {'소요시간':>15}")
print("-" * 80)
print(f"{'CF-KNN (k=30)':<20} {prec_cf:>15.6f} {rec_cf:>15.6f} {cf_time/60:>14.2f}분")
print(f"{'Content-Based':<20} {prec_content:>15.6f} {rec_content:>15.6f} {content_time/60:>14.2f}분")
print(f"{'Hybrid (0.6+0.4)':<20} {prec_hybrid:>15.6f} {rec_hybrid:>15.6f} {hybrid_time/60:>14.2f}분")
print("-" * 80)
print(f"{'총 소요시간':<20} {'':<15} {'':<15} {(cf_time+content_time+hybrid_time)/60:>14.2f}분")
print("-" * 80)

# ====================================================================
# 모델 성능 비교 요약
# ====================================================================
print("\n[모델 성능 순위]")
print("-" * 80)

models = [
    ("CF-KNN (k=30)", prec_cf, rec_cf),
    ("Content-Based", prec_content, rec_content),
    ("Hybrid (0.6+0.4)", prec_hybrid, rec_hybrid)
]

# Precision 기준 정렬
models_by_prec = sorted(models, key=lambda x: x[1], reverse=True)

for rank, (name, prec, rec) in enumerate(models_by_prec, 1):
    score = (prec + rec) / 2  # F1-score 개념
    print(f"{rank}위: {name:<20} | Precision: {prec:.6f} | Recall: {rec:.6f} | F1: {score:.6f}")

print("-" * 80)

# ====================================================================
# 필터링 정보 요약
# ====================================================================
print("\n[필터링 정보]")
print("-" * 80)
print(f"인기도 필터링: 상위 80% (평가 수 >= {popularity_threshold:.0f}개)")
print(f"평점 필터링: 상위 90% (평균 평점 >= {rating_threshold:.2f}점)")
print(f"최종 평가 후보: {len(candidate_books_filtered)} / {len(all_books)} 책 ({len(candidate_books_filtered)/len(all_books)*100:.1f}%)")
print(f"예상 속도 향상: {len(all_books)/len(candidate_books_filtered):.1f}배 빠름")
print("-" * 80)


[순위 기반 평가 - Ranking Evaluation]
평가 사용자 (샘플링): 1000 / 4927 (20.3%)
샘플링 신뢰도: ~98% (표준오차 <1%)
전체 후보 책: 8678
인기도 필터링: 8678 → 4429 책 (51.0%)
평점 필터링: 8678 → 4339 책 (50.0%)
최종 후보 책: 8678 → 2278 책 (26.3%)
예상 속도 향상: 3.8배

CF-KNN Top-K 생성 중...
CF-KNN [██████████████████████████████] 100.00% (1000/1000) | 예상 남은시간: 0.0분
✓ CF-KNN 완료 (33.89분)

컨텐트 기반 Top-K 생성 중...
Content [██████████████████████████████] 100.00% (1000/1000) | 예상 남은시간: 0.0분
✓ Content-Based 완료 (29.14분)

하이브리드 Top-K 생성 중...
Hybrid [██████████████████████████████] 100.00% (1000/1000) | 예상 남은시간: 0.0분
✓ Hybrid 완료 (84.15분)

Precision & Recall 계산 중...

[Ranking 결과 @ K=10]
--------------------------------------------------------------------------------
모델                      Precision@10       Recall@10            소요시간
--------------------------------------------------------------------------------
CF-KNN (k=30)               0.000061        0.000061          33.89분
Content-Based               0.000041        0.000041          29.14분
Hybri

In [21]:
# ====================================================================
# 15단계: 최종 요약 및 결론
# ====================================================================
print("\n[최종 요약]")
print("=" * 60)
print(f"총 사용자: {ratings_f['user_id'].nunique()}")
print(f"총 책: {ratings_f['book_id'].nunique()}")
print(f"총 상호작용: {len(ratings_f)}")
print(f"평균 밀도: {len(ratings_f) / (ratings_f['user_id'].nunique() * ratings_f['book_id'].nunique()) * 100:.4f}%")
print()
print("[평가 방법]")
print("1. Rating Prediction: RMSE, MAE (절대 평점 예측 정확도)")
print("2. Ranking Evaluation: Precision@K, Recall@K (상위 K개 추천 품질)")
print()
print("[사용된 알고리즘]")
print("1. CF-KNN: 사용자-기반 협업 필터링 (코사인 유사도)")
print("2. Co ntent-Based: TF-IDF + 숫자 피처 (제목, 저자, 태그, 평점, 언어)")
print("3. Hybrid: CF-KNN과 Content-Based의 가중합 (α=0.6)")
print()
print("[추가 컬럼 활용]")
print("- 저자, 원제, 태그 (텍스트 피처)")
print("- 평균 평점, 평점 수, 출판 연도, 언어 (숫자 피처)")
print("=" * 60)


[최종 요약]
총 사용자: 4927
총 책: 8678
총 상호작용: 414113
평균 밀도: 0.9685%

[평가 방법]
1. Rating Prediction: RMSE, MAE (절대 평점 예측 정확도)
2. Ranking Evaluation: Precision@K, Recall@K (상위 K개 추천 품질)

[사용된 알고리즘]
1. CF-KNN: 사용자-기반 협업 필터링 (코사인 유사도)
2. Co ntent-Based: TF-IDF + 숫자 피처 (제목, 저자, 태그, 평점, 언어)
3. Hybrid: CF-KNN과 Content-Based의 가중합 (α=0.6)

[추가 컬럼 활용]
- 저자, 원제, 태그 (텍스트 피처)
- 평균 평점, 평점 수, 출판 연도, 언어 (숫자 피처)
